In [ ]:
import numpy as np

import pydrake
from pydrake.all import (
    PiecewisePolynomial, RigidTransform, RotationMatrix
)

from src.drake_helpers import (
    BuildAndSimulateTrajectory, 
    setup_manipulation_station,
    visualize_transform, 
)
from src.ik import create_q_knots
from src.throw import (
    plan_pickup, 
    plan_prethrow_pose,
    add_go_to_pose_via_jointinterpolation,
    plan_throw,
)

In [ ]:
# start a single meshcat server instance to use for remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])
print(web_url)

In [ ]:
# Get initial pose of the gripper by using default context of manip station.
P_WORLD_TARGET = np.array([-4, 0, 2])
GRIPPER_TO_OBJECT_DIST = 0.13 # meters
T_world_objectInitial = RigidTransform(
    p=[-.1, -.69, 1.04998503e-01],
    R=RotationMatrix.MakeZRotation(np.pi/2.0)
)
T_world_gripperObject = RigidTransform(
    p=T_world_objectInitial.translation() + np.array([0, 0, GRIPPER_TO_OBJECT_DIST]),
    R=RotationMatrix.MakeXRotation(-np.pi/2.0)
)
T_world_robotInitial, meshcat = setup_manipulation_station(T_world_objectInitial, zmq_url)

#object frame viz
visualize_transform(meshcat, "T_world_obj0", T_world_objectInitial)

# plan the throw
t_lst, q_knots, total_time = plan_pickup(T_world_robotInitial, T_world_gripperObject)

# clear the bins via a waypoint
T_world_hackyWayPoint = RigidTransform(
    p=[-.6, -0.0, 0.6],
    R=RotationMatrix.MakeXRotation(-np.pi/2.0), #R_WORLD_PRETHROW, #RotationMatrix.MakeXRotation(-np.pi/2.0),
)
t_lst, q_knots = add_go_to_pose_via_jointinterpolation(
    T_world_robotInitial, 
    T_world_hackyWayPoint, 
    t_start=total_time,
    t_lst=t_lst,
    q_knots=q_knots,
    time_interval_s=5
)

# plan the prethrow pose
T_world_prethrow, T_world_throw = plan_prethrow_pose(
    T_world_robotInitial=T_world_robotInitial,
    p_world_target=P_WORLD_TARGET,
    gripper_to_object_dist=GRIPPER_TO_OBJECT_DIST,
    meshcat=meshcat
)

# go to prethrow
t_lst, q_knots = add_go_to_pose_via_jointinterpolation(
    T_world_hackyWayPoint, 
    T_world_prethrow, 
    t_start=total_time + 5,
    t_lst=t_lst,
    q_knots=q_knots,
    time_interval_s=5
)

# go to throw
t_lst, q_knots = add_go_to_pose_via_jointinterpolation(
    T_world_prethrow, 
    T_world_throw, 
    t_start=total_time + 5 + 5,
    t_lst=t_lst,
    q_knots=q_knots,
    time_interval_s=5,
    do_magic=True
)

# make gripper trajectory
gripper_times_lst = np.array([
    0.,
    1.0,
    0.5,
    1.0,
    5,
    5,
    5,
])
gripper_cumulative_times_lst = np.cumsum(gripper_times_lst)
GRIPPER_OPEN = 0.5
GRIPPER_CLOSED = 0.0
gripper_knots = np.array([
    GRIPPER_OPEN,
    GRIPPER_OPEN,
    GRIPPER_CLOSED,
    GRIPPER_CLOSED,
    GRIPPER_CLOSED,
    GRIPPER_CLOSED,
    GRIPPER_CLOSED,
]).reshape(1,gripper_times_lst.shape[0])
g_traj = PiecewisePolynomial.FirstOrderHold(gripper_cumulative_times_lst, gripper_knots)

In [ ]:
# turn trajectory into joint space
q_knots = np.array(q_knots)
q_traj = PiecewisePolynomial.CubicShapePreserving(t_lst, q_knots[:, 0:7].T)

In [ ]:
# do the thing
simulator, station_plant, meshcat = BuildAndSimulateTrajectory(q_traj, g_traj, T_world_objectInitial, None, zmq_url)
visualize_transform(
    meshcat,
    "TARGET",
    RigidTransform(RotationMatrix.MakeZRotation(0), P_WORLD_TARGET),
    prefix='',
    length=0.3,
    radius=0.02
)

meshcat.start_recording()
print(f"Running for {q_traj.end_time()} seconds")
simulator.AdvanceTo(q_traj.end_time())
meshcat.stop_recording()
meshcat.publish_recording()